## Loading data into neo4j


In [ ]:
from graphutils.utils import Neo4jConnect

connection = Neo4jConnect('admin', 'cran2graph')

# Create a constraint on 'Package' nodes based on the 'package' property
constraint_package_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Package) REQUIRE p.package IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_person_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Person) REQUIRE a.person IS UNIQUE"

# Create an index on the 'person' property of 'Person' nodes
person_index_query = "CREATE INDEX IF NOT EXISTS FOR :Person(person)"

In [ ]:
from graphutils.utils import get_neo4j_connection
db_session = get_neo4j_connection()

# Create a constraint on 'Package' nodes based on the 'package' property
constraint_package_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Package) REQUIRE p.package IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_person_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Person) REQUIRE a.person IS UNIQUE"

# Create an index on the 'person' property of 'Person' nodes
person_index_query = "CREATE INDEX IF NOT EXISTS FOR :Person(person)"

constraints = [constraint_person_query, constraint_package_query, person_index_query]

for constraint in constraints:
    print(f"Creating the constraints with: {constraint}")
    db_session.run(constraint)

In [1]:
from graphutils.utils import Neo4jConnect

connection = Neo4jConnect('admin', 'cran2graph')
connection.clean_database()
connection.close()